In [47]:
# imports
from csv import DictReader, DictWriter
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [48]:
og_column_names = ['station_id', 'longitude', 'latitude', 'time', 'AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'WaveHeight', 'WavePeriod', 'MeanWaveDirection', 'Hmax', 'AirTemperature', 'DewPoint', 'SeaTemperature']

column_names = ['AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'AirTemperature', 'SeaTemperature', 'WaveHeight', 'WavePeriod']

with open('../data/raw_data.csv', 'r') as f1, open('../data/clean_data.csv', 'w') as f2:
    reader = DictReader(f1, fieldnames=og_column_names)
    writer = DictWriter(f2, fieldnames=column_names, lineterminator='\n')

    next(reader)
    next(reader)

    writer.writeheader()

    for line in tqdm(reader):
        if any(line[column_name] == 'NaN' for column_name in column_names):
            continue
        writer.writerow({column_name: line[column_name] for column_name in column_names})


613392it [00:02, 275753.91it/s]


In [49]:
df = pd.read_csv('../data/clean_data.csv')
df.describe()

,AtmospherePressure,WindDirection,WindSpeed,Gust,AirTemperature,SeaTemperature,WaveHeight,WavePeriod
count,417645.000000,417645.000000,417645.000000,417645.000000,417645.000000,417645.000000,417645.000000,417645.000000
mean,1013.734783,206.227090,15.065121,21.330294,11.625455,12.446026,2.316566,6.131247
std,12.241317,89.490168,6.882371,9.552581,2.882423,2.302647,1.537627,1.734126
min,910.300000,0.000000,0.000000,0.000000,0.264000,0.200000,0.100000,2.000000
25%,1006.400000,150.000000,10.000000,14.231000,9.500000,10.700000,1.200000,5.000000
50%,1015.000000,220.000000,15.000000,20.000000,11.600000,12.300000,2.000000,6.000000
75%,1022.200000,271.000000,19.639000,27.000000,14.000000,14.355000,3.000000,7.000000
max,1046.400000,360.000000,76.506000,200.000000,22.104000,24.200000,15.742000,18.000000


In [50]:
# Use the interquartile range to remove the outliers in data.
for column_name in column_names:

    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    lower_whisker = Q1 - 1.5*IQR
    upper_whisker = Q3 + 1.5*IQR
    df = df[(df[column_name] >= lower_whisker) & (df[column_name] <= upper_whisker)]

df.describe()

,AtmospherePressure,WindDirection,WindSpeed,Gust,AirTemperature,SeaTemperature,WaveHeight,WavePeriod
count,391691.000000,391691.000000,391691.000000,391691.000000,391691.000000,391691.000000,391691.000000,391691.000000
mean,1014.709762,204.217605,14.461454,20.359107,11.734562,12.515255,2.090284,5.939886
std,11.267965,90.336266,6.303322,8.473279,2.881194,2.331295,1.192423,1.550460
min,982.700000,0.000000,0.000000,0.000000,2.861000,5.900000,0.100000,2.000000
25%,1007.458000,150.000000,10.000000,14.000000,9.600000,10.700000,1.200000,5.000000
50%,1015.600000,215.000000,14.000000,20.000000,11.800000,12.500000,1.875000,6.000000
75%,1022.600000,270.000000,19.000000,25.473000,14.106000,14.424000,2.800000,7.000000
max,1045.800000,360.000000,32.788000,46.450000,20.800000,19.200000,5.600000,10.000000


In [51]:
features = column_names[:-2]
labels = column_names[-2:]

X = df[features].to_numpy()
# Note that we have 2 labels
Y = df[labels].to_numpy()

In [52]:
# The commonly used split of 75% for training, 10% for validation, and 15%
# for the test sets has been followed when splitting the dataset.
# V Roshan Joseph. Optimal ratio for data splitting. Statistical Analysis and Data Mining: The
# ASA Data Science Journal, 15(4):531–538, 2022.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.6, random_state=42)

# y1: WaveHeight
y1_train = Y_train[:,0]
y1_val = Y_val[:,0]
y1_test = Y_test[:,0]

# y2: WavePeriod
y2_train = Y_train[:,1]
y2_val = Y_val[:,1]
y2_test = Y_test[:,1]

In [54]:
# Linear Regression
regression1 = LinearRegression()
regression1.fit(X_train, y1_train)

y1_pred = regression1.predict(X_test)
test_error = mean_squared_error(y1_test, y1_pred)
print('The testing error is: ', test_error)    # print the training error
print("w1 = ", regression1.coef_)   # print the learnt w1
print("w0 = ",regression1.intercept_) # print the learnt w0

The testing error is:  0.8161253516700292
w1 =  [-0.00935662  0.00096564 -0.12210126  0.16731828 -0.0005244  -0.02087526]
w0 =  10.013024702085218
